In [1]:
import pandas as pd
import numpy as np
import itertools 
import matplotlib.pyplot as plt
import random
random.seed(3)

In [50]:
users = pd.read_csv(
    'Assignment_1/ml-1m/users.dat',
    sep = "::",
    names = ['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'],
    encoding='latin-1',
    engine='python',
)

users.head()

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [3]:
movies = pd.read_csv(
    'Assignment_1/ml-1m/movies.dat',
    sep = "::",
    names = ['MovieID', 'Title', 'Genres'],
    encoding='latin-1',
    engine='python',
)

movies.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings = pd.read_csv(
    'Assignment_1/ml-1m/ratings.dat',
    sep = "::",
    names = ['UserID', 'MovieID', 'Rating', 'Timestamp'],
    encoding='latin-1',
    engine='python',
)

ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
#MATRIX FACTORIZATION WITHOUT CROSS VALIDATION

learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

x = ratings.pivot(index='MovieID', columns='UserID', values='Rating')
M=np.random.rand(x.shape[0],num_factors)
M=pd.DataFrame(M, index=x.index)
U=np.random.rand(num_factors,x.shape[1])
U=pd.DataFrame(U, columns=x.columns)

indexes=x[x.notnull()].stack().index
iterat=1
rmse=100
newrmse=99

while (iterat < num_iter and newrmse-rmse < 0.01 ):
    print('iteration: ' + str(iterat))
    iterat += 1
    for k in indexes:
        movie = M.loc[k[0],]
        user = U.loc[:,k[1]]
        x_hat = np.dot(movie,user)
        error = x.loc[k[0],k[1]] - x_hat
        M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie)
        U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user)
            
    factoriz=pd.DataFrame(np.matmul(M, U), index=x.index, columns= x.columns)
    rmse=newrmse
    newrmse=np.sqrt(((factoriz-x)**2).mean().mean())
            
    #rmse = np.sqrt(np.mean((np.matmul(M, U)-train)**2))
mae = abs(factoriz-x).mean().mean()


iteration: 1


KeyboardInterrupt: 

In [ ]:
#MATRIX FACTORIZATION
# google colab / runtime - cambia tipo - GPU/TPU
x = ratings.sample(frac = 1)
X = np.array_split(x, 5)
learn_rate=0.005
reg=0.05
num_factors=10
num_iter=75

rmses = []
maes = []

for i in range(1):
    X_test = X[i]
    X_train = pd.concat([X[j] for j in range(5) if j!=i])
    train=X_train.pivot(index='MovieID', columns='UserID', values='Rating')
    test = X_test.pivot(index='MovieID', columns='UserID', values='Rating')
    
    M=np.random.rand(train.shape[0],num_factors)
    M=pd.DataFrame(M, index=train.index)
    U=np.random.rand(num_factors,train.shape[1])
    U=pd.DataFrame(U, columns=train.columns)
    
    indexes=train[train.notnull()].stack().index
    iterat=1
    rmse=100
    newrmse=99
    
    while (iterat < num_iter and newrmse-rmse < 0.01 ):
        print('Set: ' + str(i) + ', iteration: ' + str(iterat))
        iterat += 1
        for k in indexes:
            movie = M.loc[k[0],]
            user = U.loc[:,k[1]]
            x_hat = np.dot(movie,user)
            error = train.loc[k[0],k[1]] - x_hat
            M.loc[k[0],] = movie + learn_rate * (2*error*user - reg*movie)
            U.loc[:,k[1]] = user + learn_rate * (2*error*movie - reg*user)
            
        factoriz=pd.DataFrame(np.matmul(M, U), index=train.index, columns= train.columns)
        rmse=newrmse
        newrmse=np.sqrt(((factoriz[factoriz.columns & test.columns]-test)**2).mean().mean())
            
    #rmse = np.sqrt(np.mean((np.matmul(M, U)-train)**2))
    rmses += [newrmse]
    mae = abs(factoriz[factoriz.columns & test.columns]-test).mean().mean()
    maes += [mae]

In [43]:
genres = [s.split('|')[0] for s in movies_rated.loc[:,'Genres']]

In [48]:
# Visualize movies labelled by genre
pca = PCA(n_components=2)

PCs = pca.fit_transform(M)

plt.figure(figsize=(10,10)) 

for genre in genres:
    mask = np.array(movies_rated[['Genres']]) == genre
    mask = [mask[i][0] for i in range(len(mask))]
    plt.scatter(PCs[mask, 0], PCs[mask, 1], label=genre, s=25, alpha=1)

plt.legend(bbox_to_anchor=(1.00, 1), loc='upper left',fontsize=20)

ValueError: Image size of 605x107507 pixels is too large. It must be less than 2^16 in each direction.

<Figure size 432x288 with 1 Axes>